<a href="https://colab.research.google.com/github/KiroSafwat/NLP_workshop/blob/main/NLP_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install nltk
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [11]:



df = pd.read_csv('twitter_training.csv', header=None, encoding='utf-8')


In [12]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
print(df.columns)

Index([0, 1, 2, 3], dtype='int64')


In [14]:
import re


def remove_urls(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', str(text), flags=re.MULTILINE)

df['clean_tweet'] = df[3].apply(remove_urls)

print(df[['clean_tweet']].head())

                                         clean_tweet
0  im getting on borderlands and i will murder yo...
1  I am coming to the borders and I will kill you...
2  im getting on borderlands and i will kill you ...
3  im coming on borderlands and i will murder you...
4  im getting on borderlands 2 and i will murder ...


In [16]:
def remove_mentions(text):
    return re.sub(r'@\w+', '', text)

df['clean_tweet'] = df['clean_tweet'].apply(remove_mentions)
print(df[['clean_tweet']].head(2))

                                         clean_tweet
0  im getting on borderlands and i will murder yo...
1  I am coming to the borders and I will kill you...


In [18]:
def clean_hashtags(text):
    return re.sub(r'#', '', text)

df['clean_tweet'] = df['clean_tweet'].apply(clean_hashtags)
print(df[['clean_tweet']].head(2))

                                         clean_tweet
0  im getting on borderlands and i will murder yo...
1  I am coming to the borders and I will kill you...


In [19]:
df['clean_tweet'] = df['clean_tweet'].astype(str).str.lower()
print(df[['clean_tweet']].head(2))

                                         clean_tweet
0  im getting on borderlands and i will murder yo...
1  i am coming to the borders and i will kill you...


In [20]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df['clean_tweet'] = df['clean_tweet'].apply(remove_punctuation)
print(df[['clean_tweet']].head(2))

                                         clean_tweet
0  im getting on borderlands and i will murder yo...
1  i am coming to the borders and i will kill you...


In [21]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

df['clean_tweet'] = df['clean_tweet'].apply(remove_numbers)
print(df[['clean_tweet']].head(2))

                                         clean_tweet
0  im getting on borderlands and i will murder yo...
1  i am coming to the borders and i will kill you...


In [24]:

import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:


df['tokens'] = df['clean_tweet'].apply(lambda x: x.split())

print("--- 2. Tokenization Complete ---")
print(df[['tokens']].head(2))

--- 2. Tokenization Complete ---
                                              tokens
0  [im, getting, on, borderlands, and, i, will, m...
1  [i, am, coming, to, the, borders, and, i, will...


In [33]:

from sklearn.feature_extraction.text import TfidfVectorizer

df['final_text'] = df['tokens'].apply(lambda x: ' '.join(x))

Y = df[2]


tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 3)
)
X = tfidf_vectorizer.fit_transform(df['final_text']).toarray()

print(f"--- 4. Vectorization (TF-IDF) Complete ---")
print(f"Feature matrix shape (X): {X.shape}")

--- 4. Vectorization (TF-IDF) Complete ---
Feature matrix shape (X): (74682, 5000)


In [34]:

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42,)

print("--- 5. Train-Test Split Complete ---")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

--- 5. Train-Test Split Complete ---
X_train shape: (59745, 5000)
X_test shape: (14937, 5000)


In [35]:
# Import the Logistic Regression model and evaluation metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# --- 6. MODEL TRAINING ---

# 1. Initialize the model
# We set max_iter to a higher value for convergence on larger datasets
model = LogisticRegression(max_iter=1000, random_state=42)

# 2. Train the model using the training data
model.fit(X_train, Y_train)

print("--- Model Training Complete ---")

# 3. Make predictions on the test set
Y_pred = model.predict(X_test)

# 4. Evaluate the model
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)

print(f"\nAccuracy Score: {accuracy:.4f}")
print("\nClassification Report (Detailed Results):")
print(report)

--- Model Training Complete ---

Accuracy Score: 0.6649

Classification Report (Detailed Results):
              precision    recall  f1-score   support

  Irrelevant       0.64      0.49      0.55      2592
    Negative       0.71      0.76      0.73      4519
     Neutral       0.60      0.61      0.61      3596
    Positive       0.68      0.72      0.70      4230

    accuracy                           0.66     14937
   macro avg       0.66      0.64      0.65     14937
weighted avg       0.66      0.66      0.66     14937

